In [ ]:
from pathlib import Path
Path("/content").mkdir(parents=True, exist_ok=True)

In [ ]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

In [ ]:
!ls

In [ ]:
%pip install torchsde einops diffusers accelerate xformers==0.0.27
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/flux1-schnell.safetensors -d /content/TotoroUI/models/unet -o flux1-schnell.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors



In [ ]:
import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-schnell.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

In [ ]:
with torch.inference_mode():
    positive_prompt = "A cryptopunk pixelart nft of a tiger that has fingerless gloves, crossbow, saber. Style is pixelart cryptopunk, so only the upper part of their body is visible, it has a tiger pattern, and it has a punk style. The background is homogeneous. The portrait is visible to the bottom of the image. The tiger is looking at the viewer with a fierce expression. The image is like a portrait to the bottom of the artpiece, not an icon. It is not a bust. The tiger is facing directly towards the viewer in a frontal pose. Its head is held high, giving an impression of confidence or coolness. The shoulders are visible, suggesting an upright, almost human-like posture. It has a fingerless gloves, crossbow, saber."
    width = 1024
    height = 1024
    seed = 0
    steps = 4
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0] 
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

In [ ]:
def image_inference(positive_prompt,steps):
    with torch.inference_mode():
        positive_prompt=positive_prompt
        width = 1024
        height = 1024
        seed = 0
        steps = steps
        sampler_name = "euler"
        scheduler = "simple"

        if seed == 0:
            seed = random.randint(0, 18446744073709551615)
        print(seed)

        cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
        cond = [[cond, {"pooled_output": pooled}]]
        noise = RandomNoise.get_noise(seed)[0] 
        guider = BasicGuider.get_guider(unet, cond)[0]
        sampler = KSamplerSelect.get_sampler(sampler_name)[0]
        sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
        latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
        sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
        model_management.soft_empty_cache()
        decoded = VAEDecode.decode(vae, sample)[0].detach()
        Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")

        

In [1]:
import requests
import json

while True:
    #get prompt from url: 
    url = "http://127.0.0.1:5001/prompt"
    response = requests.get(url)
    prompt = response.json()
    print(prompt['prompt'])
    image_inference(prompt['prompt'],4)

    #send image to url
    url = "http://127.0.0.1:5001/upload"
    files = {'file': open('/content/flux.png', 'rb')}
    response = requests.post(url, files=files)
    print(response.json())
    






A cryptopunk pixelart nft of a tiger that has long coat, energy sword, battering ram. Style is pixelart cryptopunk, so only the upper part of their body is visible, it has a tiger pattern, and it has a punk style. The background is homogeneous. The portrait is visible to the bottom of the image. The tiger is looking at the viewer with a fierce expression. The image is like a portrait to the bottom of the artpiece, not an icon. It is not a bust. The tiger is facing directly towards the viewer in a frontal pose. Its head is held high, giving an impression of confidence or coolness. The shoulders are visible, suggesting an upright, almost human-like posture. It has a long coat, energy sword, battering ram.
A cryptopunk pixelart nft of a tiger that has saber, diving mask, halberd. Style is pixelart cryptopunk, so only the upper part of their body is visible, it has a tiger pattern, and it has a punk style. The background is homogeneous. The portrait is visible to the bottom of the image. T

KeyboardInterrupt: 